In [13]:
import pdb

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from torchtext.vocab import GloVe
from torchtext.data import Field, BucketIterator
from torchtext.data.metrics import bleu_score
import spacy

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
#Reading files

with open('trainen.txt', encoding='utf8') as f:
    eng_train = list(map(lambda x: x.rstrip(), f.readlines()))
    
with open('trainta.txt', encoding='utf8') as f:
    tamil_train = list(map(lambda x: x.rstrip(), f.readlines()))
    
with open('deven.txt', encoding='utf8') as f:
    eng_test = list(map(lambda x: x.rstrip(), f.readlines()))
    
with open('devta.txt', encoding='utf8') as f:
    tamil_test = list(map(lambda x: x.rstrip(), f.readlines()))
    
embedding_glove = GloVe(name='6B', dim=100)

spacy_en = spacy.load('en_core_web_sm')

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

ENG = Field(tokenize = tokenize_en, init_token='sos', eos_token = 'eos', lower=True)
processed_eng_train = list(map(lambda x: ENG.preprocess(x), eng_train))
processed_eng_test = list(map(lambda x: ENG.preprocess(x), eng_test))

ENG.build_vocab(processed_eng_train, vectors=embedding_glove)

In [5]:
def preprocess(processed_eng):
    
    #function to return the numericalized version of the tokenized sentences
    X = []
    for tokenized_sentence in processed_eng:
        int_sequence = [2]  #first element is the SOS token 
        for token in tokenized_sentence:
            int_sequence.append(ENG.vocab.stoi[token])
        int_sequence.append(3) #last element is the EOS token
        X.append(int_sequence)
    
    return X

# X_train and X_test are lists of lists with the integer sequences for a given sentence
X_train = preprocess(processed_eng_train)
X_test = preprocess(processed_eng_test)

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Same thing for Tamil sentences
TAM = Tokenizer()
TAM.fit_on_texts(tamil_train)
Y_train = TAM.texts_to_sequences(tamil_train)
Y_test = TAM.texts_to_sequences(tamil_test)

#adding EOS token
_ = [y.append(0) for y in Y_train]
_ = [y.append(0) for y in Y_test]

Using TensorFlow backend.


In [7]:
source_vocab_size = len(ENG.vocab)
target_vocab_size = len(TAM.word_index)+1
print(source_vocab_size)
print(target_vocab_size)

9736
18669


In [8]:
class Encoder(nn.Module):
    
    def __init__(self, embed_size, hidden_size, ENG):
        
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.embed = nn.Embedding.from_pretrained(ENG.vocab.vectors)
        self.lstm = nn.LSTM(embed_size, hidden_size)
        
    def forward(self, x, enc_hidden):
        
        enc_output, enc_hidden = self.lstm(self.embed(x).view(1,1,-1), enc_hidden)
        return enc_output, enc_hidden

In [18]:
class Decoder(nn.Module):
    
    def __init__(self, embed_size, hidden_size, n_classes):
        
        super(Decoder, self).__init__()
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.embed = nn.Embedding(n_classes, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size)
        self.fc = nn.Linear(hidden_size, n_classes)
        self.logsoftmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x, dec_hidden):
        
        dec_output, dec_hidden = self.lstm(self.embed(x).view(1,1,-1), dec_hidden)
        dec_output = self.logsoftmax(self.fc(dec_output[0]))
        return dec_output, dec_hidden

In [58]:
def loss_per_pair(source_sentence, target_sentence, enc_obj, dec_obj, enc_optimiser, dec_optimiser, loss_fn):
    
    Ts = source_sentence.size(-1)
    Tt = target_sentence.size(-1)
    enc_hidden = (torch.zeros(1, 1, enc_obj.hidden_size, device=device), torch.zeros(1, 1, enc_obj.hidden_size, device=device))
    
    enc_optimiser.zero_grad()
    dec_optimiser.zero_grad()
    
    loss_val = 0
    
    for i in range(Ts):
        enc_output, enc_hidden = enc_obj(source_sentence[i], enc_hidden)
    
    dec_input = torch.tensor([[2]], device=device)  #SOS token
    
    # first hidden state of decoder is made the final hidden state of the encoder and cell state is initialised with zeros
    dec_hidden = (enc_hidden[0], torch.zeros(1, 1, dec_obj.hidden_size, device=device))
    
    for i in range(Tt):
        
        dec_output, dec_hidden = dec_obj(dec_input, dec_hidden)
        _ , index = dec_output.topk(1)
        dec_input = index.squeeze().detach()
            
        target_word = torch.cuda.LongTensor([target_sentence[i].item()])
        
        loss_val += loss_fn(dec_output, target_word)
        
        if dec_input.item() == 0:
            break

    loss_val.backward()
    
    nn.utils.clip_grad_norm_(enc_obj.parameters(), 0.5, 1)
    nn.utils.clip_grad_norm_(dec_obj.parameters(), 0.5, 1)

    enc_optimiser.step()
    dec_optimiser.step()

    return loss_val.item()/Tt

In [62]:
def train_model(sources, targets, enc_obj, dec_obj):
    
    loss_fn = nn.NLLLoss()
    enc_optimiser = optim.SGD(enc_obj.parameters(), lr=0.001, momentum=0.9)
    dec_optimiser = optim.SGD(dec_obj.parameters(), lr=0.001, momentum=0.9)
#     enc_optimiser = optim.Adagrad(enc_obj.parameters())
#     dec_optimiser = optim.Adagrad(dec_obj.parameters())
    
    max_epochs = 100
    old_loss = np.inf
    indices = [i for i in range(len(sources))]
    
    for epoch in range(max_epochs):
        
        # shuffling the input data manually at the start of every new epoch
        np.random.shuffle(indices)
        sources = list(np.array(sources)[indices])
        targets = list(np.array(targets)[indices])
        
        running_loss = 0.0
        num_sentences = len(sources)
        
        for i in range(num_sentences):
            source_sentence = torch.cuda.LongTensor(sources[i])
            target_sentence = torch.cuda.LongTensor(targets[i])

            loss = loss_per_pair(source_sentence, target_sentence, enc_obj, dec_obj, enc_optimiser, dec_optimiser, loss_fn)
            running_loss += loss

            if i%int(num_sentences*0.1)==0:
                print("Epoch", epoch+1, ":", (i/int(num_sentences*0.1))*10,'% done')
                print("Current loss:", running_loss)
                
        print('\nEpoch', epoch+1,"\n")
        print("Encoder obj lstm wts sum=", torch.sum(enc_obj.lstm.weight_ih_l0))
        print("Encoder obj lstm bias sum=", torch.sum(enc_obj.lstm.bias_ih_l0))
        print('Decoder obj emb wts sum=', torch.sum(dec_obj.embed.weight.data))
        print('Decoder obj lstm wts sum=', torch.sum(dec_obj.lstm.weight_ih_l0.data))
        print('Decoder obj lstm bias sum=', torch.sum(dec_obj.lstm.bias_ih_l0.data))
        print('Decoder obj linear wts sum=', torch.sum(dec_obj.fc.weight.data))
        print('Decoder obj linear bias sum=', torch.sum(dec_obj.fc.bias.data))
        
        
        if abs(running_loss-old_loss)/running_loss < 1e-3:
            print('Converged')
            break
    
        old_loss = running_loss

    print("Finished Training")

In [63]:
embedding_size = 100
hidden_size = 150
encoder = Encoder(embedding_size, hidden_size, ENG).to(device)
decoder = Decoder(embedding_size, hidden_size, target_vocab_size).to(device)
train_model(X_train, Y_train, encoder, decoder)

Epoch 1 : 0.0 % done
Current loss: 9.82754135131836
Epoch 1 : 10.0 % done
Current loss: 25755.683479615316
Epoch 1 : 20.0 % done
Current loss: 51502.66197272957
Epoch 1 : 30.0 % done
Current loss: 77215.21324324333
Epoch 1 : 40.0 % done
Current loss: 102899.94318143725
Epoch 1 : 50.0 % done
Current loss: 121611.55016383945
Epoch 1 : 60.0 % done
Current loss: 133938.8418479452
Epoch 1 : 70.0 % done
Current loss: 146497.32315648973
Epoch 1 : 80.0 % done
Current loss: 158783.32897929766
Epoch 1 : 90.0 % done
Current loss: 171805.28109355972
Epoch 1 : 100.0 % done
Current loss: 196143.57038203054

Epoch 1 

Encoder obj lstm wts sum= tensor(5.1913, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(0.8888, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(1194.8718, device='cuda:0')
Decoder obj lstm wts sum= tensor(-15.5397, device='cuda:0')
Decoder obj lstm bias sum= tensor(-0.5211, device='cuda:0')
Decoder obj linear wts sum= tensor(48.2973, 

Decoder obj lstm bias sum= tensor(0.1383, device='cuda:0')
Decoder obj linear wts sum= tensor(54.1911, device='cuda:0')
Decoder obj linear bias sum= tensor(-4.5419, device='cuda:0')
Epoch 9 : 0.0 % done
Current loss: 4.986844062805176
Epoch 9 : 10.0 % done
Current loss: 11941.203287185077
Epoch 9 : 20.0 % done
Current loss: 23822.639320536295
Epoch 9 : 30.0 % done
Current loss: 35845.762511963556
Epoch 9 : 40.0 % done
Current loss: 47726.2158771547
Epoch 9 : 50.0 % done
Current loss: 59526.074597624705
Epoch 9 : 60.0 % done
Current loss: 71294.32431022631
Epoch 9 : 70.0 % done
Current loss: 83189.1776628766
Epoch 9 : 80.0 % done
Current loss: 95074.94133035587
Epoch 9 : 90.0 % done
Current loss: 106861.63337565398
Epoch 9 : 100.0 % done
Current loss: 118753.40509349116

Epoch 9 

Encoder obj lstm wts sum= tensor(6.4848, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(1.8619, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(1194.8347, d

Epoch 16 : 80.0 % done
Current loss: 88041.95402068774
Epoch 16 : 90.0 % done
Current loss: 98851.52716449038
Epoch 16 : 100.0 % done
Current loss: 109692.75820816717

Epoch 16 

Encoder obj lstm wts sum= tensor(8.3522, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(3.9472, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(1194.8342, device='cuda:0')
Decoder obj lstm wts sum= tensor(-11.8367, device='cuda:0')
Decoder obj lstm bias sum= tensor(1.4868, device='cuda:0')
Decoder obj linear wts sum= tensor(64.7599, device='cuda:0')
Decoder obj linear bias sum= tensor(-2.2221, device='cuda:0')
Epoch 17 : 0.0 % done
Current loss: 3.5951175689697266
Epoch 17 : 10.0 % done
Current loss: 10998.101140331793
Epoch 17 : 20.0 % done
Current loss: 21806.48338822661
Epoch 17 : 30.0 % done
Current loss: 32704.532314967208
Epoch 17 : 40.0 % done
Current loss: 43651.725291980416
Epoch 17 : 50.0 % done
Current loss: 54519.81410588833
Epoch 17 : 60.0 % don

Epoch 24 : 10.0 % done
Current loss: 10452.018523840125
Epoch 24 : 20.0 % done
Current loss: 20727.797875684104
Epoch 24 : 30.0 % done
Current loss: 31158.921863480326
Epoch 24 : 40.0 % done
Current loss: 41505.39982411048
Epoch 24 : 50.0 % done
Current loss: 52075.93043320931
Epoch 24 : 60.0 % done
Current loss: 62503.8879984145
Epoch 24 : 70.0 % done
Current loss: 72763.03322129507
Epoch 24 : 80.0 % done
Current loss: 83040.14428902388
Epoch 24 : 90.0 % done
Current loss: 93421.43326177093
Epoch 24 : 100.0 % done
Current loss: 103683.25614448688

Epoch 24 

Encoder obj lstm wts sum= tensor(9.2946, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(5.7152, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(1194.8435, device='cuda:0')
Decoder obj lstm wts sum= tensor(-10.3851, device='cuda:0')
Decoder obj lstm bias sum= tensor(2.3491, device='cuda:0')
Decoder obj linear wts sum= tensor(76.6861, device='cuda:0')
Decoder obj linear bias sum= 

Epoch 32 : 10.0 % done
Current loss: 10280.010339053297
Epoch 32 : 20.0 % done
Current loss: 20439.451908669424
Epoch 32 : 30.0 % done
Current loss: 30483.432123664734
Epoch 32 : 40.0 % done
Current loss: 40650.452304095714
Epoch 32 : 50.0 % done
Current loss: 50812.0108677405
Epoch 32 : 60.0 % done
Current loss: 60804.370842420314
Epoch 32 : 70.0 % done
Current loss: 70918.06112625563
Epoch 32 : 80.0 % done
Current loss: 80886.86160440858
Epoch 32 : 90.0 % done
Current loss: 90975.02440025925
Epoch 32 : 100.0 % done
Current loss: 101127.17700201955

Epoch 32 

Encoder obj lstm wts sum= tensor(9.3937, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(6.7599, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(1194.8569, device='cuda:0')
Decoder obj lstm wts sum= tensor(-9.5624, device='cuda:0')
Decoder obj lstm bias sum= tensor(2.8916, device='cuda:0')
Decoder obj linear wts sum= tensor(89.5017, device='cuda:0')
Decoder obj linear bias sum=

Epoch 40 : 10.0 % done
Current loss: 10024.503948352292
Epoch 40 : 20.0 % done
Current loss: 19871.558647847476
Epoch 40 : 30.0 % done
Current loss: 29716.428979952598
Epoch 40 : 40.0 % done
Current loss: 39409.90665570761
Epoch 40 : 50.0 % done
Current loss: 49229.78928433847
Epoch 40 : 60.0 % done
Current loss: 59188.283806162515
Epoch 40 : 70.0 % done
Current loss: 69085.68222119872
Epoch 40 : 80.0 % done
Current loss: 79181.65546703547
Epoch 40 : 90.0 % done
Current loss: 89004.83447225345
Epoch 40 : 100.0 % done
Current loss: 98999.28758950782

Epoch 40 

Encoder obj lstm wts sum= tensor(9.3078, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(7.7769, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(1194.8683, device='cuda:0')
Decoder obj lstm wts sum= tensor(-8.7260, device='cuda:0')
Decoder obj lstm bias sum= tensor(3.4110, device='cuda:0')
Decoder obj linear wts sum= tensor(103.1735, device='cuda:0')
Decoder obj linear bias sum=

Epoch 48 : 10.0 % done
Current loss: 9825.608603630297
Epoch 48 : 20.0 % done
Current loss: 19547.126517378794
Epoch 48 : 30.0 % done
Current loss: 29392.31757745523
Epoch 48 : 40.0 % done
Current loss: 39088.53118128643
Epoch 48 : 50.0 % done
Current loss: 48790.79336672832
Epoch 48 : 60.0 % done
Current loss: 58455.28207784661
Epoch 48 : 70.0 % done
Current loss: 68193.387357955
Epoch 48 : 80.0 % done
Current loss: 77924.77637233684
Epoch 48 : 90.0 % done
Current loss: 87693.34763979711
Epoch 48 : 100.0 % done
Current loss: 97574.88418821339

Epoch 48 

Encoder obj lstm wts sum= tensor(8.9549, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(8.4696, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(1194.8773, device='cuda:0')
Decoder obj lstm wts sum= tensor(-8.0870, device='cuda:0')
Decoder obj lstm bias sum= tensor(3.8113, device='cuda:0')
Decoder obj linear wts sum= tensor(117.1413, device='cuda:0')
Decoder obj linear bias sum= tens

Epoch 56 : 10.0 % done
Current loss: 9628.952226901125
Epoch 56 : 20.0 % done
Current loss: 19317.222632047055
Epoch 56 : 30.0 % done
Current loss: 28875.21795204532
Epoch 56 : 40.0 % done
Current loss: 38621.163895386446
Epoch 56 : 50.0 % done
Current loss: 48293.821551196124
Epoch 56 : 60.0 % done
Current loss: 58066.45426771823
Epoch 56 : 70.0 % done
Current loss: 67710.18284539651
Epoch 56 : 80.0 % done
Current loss: 77299.56836222831
Epoch 56 : 90.0 % done
Current loss: 86900.13507563717
Epoch 56 : 100.0 % done
Current loss: 96620.65056345562

Epoch 56 

Encoder obj lstm wts sum= tensor(8.4228, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(8.8779, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(1194.8844, device='cuda:0')
Decoder obj lstm wts sum= tensor(-7.6459, device='cuda:0')
Decoder obj lstm bias sum= tensor(4.0924, device='cuda:0')
Decoder obj linear wts sum= tensor(131.0668, device='cuda:0')
Decoder obj linear bias sum= 

In [64]:
torch.save(encoder.state_dict(), 'encoder_weights.pt')
torch.save(decoder.state_dict(), 'decoder_weights.pt')

In [71]:
def test_loss(sources, targets, enc_obj, dec_obj):
    
    loss_fn = nn.NLLLoss()
    num_sentences = len(sources)
    total_loss = 0
        
    for i in range(num_sentences):

        with torch.no_grad():

            source_sentence = torch.cuda.LongTensor(sources[i])
            target_sentence = torch.cuda.LongTensor(targets[i])
            Ts = source_sentence.size(-1)
            Tt = target_sentence.size(-1)
            enc_hidden = (torch.zeros(1, 1, enc_obj.hidden_size, device=device), torch.zeros(1, 1, enc_obj.hidden_size, device=device))
            
            loss_val = 0
            
            for i in range(Ts):
                enc_output, enc_hidden = enc_obj(source_sentence[i], enc_hidden)

            dec_input = torch.tensor([[2]], device=device)  #SOS token

            # first hidden state of decoder is made the final hidden state of the encoder and cell state is initialised with zeros
            dec_hidden = (enc_hidden[0], torch.zeros(1, 1, dec_obj.hidden_size, device=device))

            for i in range(60):

                dec_output, dec_hidden = dec_obj(dec_input, dec_hidden)
                _ , index = dec_output.topk(1)
                dec_input = index.squeeze().detach()
                if index.item() == 0:
                    break

                target_word = torch.cuda.LongTensor([target_sentence[i].item()])

                loss_val += loss_fn(dec_output, target_word)
            
            total_loss += loss_val.item()/Tt
    
    return total_loss 

In [72]:
test_loss(X_test, Y_test, encoder, decoder)

262.6952312039345

In [73]:
def eval_bleu(enc_obj, dec_obj, source_sentence, target, target_vocab_dict):
    
    # function to return the BLEU score for a single sentence 
    
    with torch.no_grad():
        
        Ts = source_sentence.size(-1)
        enc_hidden = (torch.zeros(1, 1, enc_obj.hidden_size, device=device), torch.zeros(1, 1, enc_obj.hidden_size, device=device))

        for i in range(Ts):
            enc_output, enc_hidden = enc_obj(source_sentence[i], enc_hidden)

        dec_input = torch.tensor([[2]], device=device)  # SOS

        dec_hidden = (enc_hidden[0], torch.zeros(1, 1, dec_obj.hidden_size, device=device))

        predicted = []

        for i in range(60):
            dec_output, dec_hidden = dec_obj(dec_input, dec_hidden)
            _ , index = dec_output.data.topk(1)
            if index.item() == 0:
                #decoded_words.append('<EOS>')
                break
            else:
                predicted.append(target_vocab_dict[index.item()])

            dec_input = index.squeeze().detach()

    print(predicted)
    print(target)
    return bleu_score([predicted], [[target]])

In [67]:
def evaluate_model(enc_obj, dec_obj, source_test, target_test, target_vocab_dict):
    
    # returns the average bleu score for the model with the given test data
    
    total_bleu = 0
    for i in range(len(source_test)):
        source_sentence = torch.cuda.LongTensor(source_test[i])
        target = [target_vocab_dict[x] for x in target_test[i][:-1]]
        bleu = eval_bleu(enc_obj, dec_obj, source_sentence, target, target_vocab_dict)
        total_bleu += bleu
    
    return total_bleu/len(source_test)

In [74]:
evaluate_model(encoder, decoder, X_test, Y_test, TAM.index_word)

['நான்']
['நீங்கள்', 'எங்களுக்கு', 'கோப்ரா', 'என்ற', 'சொல்கிறேன்', 'அல்லது', 'die', 'நீ', 'நம்', 'மாஸ்டர்', 'கொல்ல', 'அதே', 'வாள்']
['நான்']
['அவர்கள்', 'அதே', 'படுக்கை', 'அதே', 'எழுந்து', 'அதே', 'மன', 'ஒரு', 'எரிவாயு', 'நிலையம்']
['நான்']
['என்ன', 'அன்று', 'இரவு', 'என்ன', 'நடந்தது', 'மறக்க', 'ஒரே', 'ஒரு', 'கணம்', 'இருந்தது']
['நான்']
['நீல்ஸ்', 'வேண்டும்', 'அடுத்த', 'எப்படியோ', 'நான்', 'பழைய', 'கொட்டகையின்', 'வேண்டும்', 'எல்', 'செய்']
['நான்']
['செம்மை', 'எந்த', 'அதிர்ஷ்டம்', 'இருந்தது', 'அதனால்', 'அவர்கள்', 'நமக்கு', 'கேட்டார்', 'இது', 'ஒரு', 'கிராக்', 'எடுத்து']
['நான்']
['நீங்கள்', 'அவரை', 'ஒரு', 'பெயர்', 'கொடுத்து', 'இருந்தால்', 'அவர்', 'ஒருவேளை', 'நல்ல', 'என்றார்', 'நீங்கள்', 'செல்லலாம்']
['நான்']
['போன்ற', 'இல்லை', 'இந்த', 'நாட்கள்', 'ஒன்று', 'நீங்கள்', 'போகிறேன்', 'கற்று', 'கொள்ள', 'வேண்டும்']
['நான்']
['நான்', 'தான்', 'பெற', 'முடியும்', 'அந்த', 'விமானம்', '8', '00', 'வரை', 'விட்டு', 'இல்லை']
['நான்']
['நான்', 'அது', 'என்று', 'எனக்கு', 'தெரியாது', 'நான்', 'ஆனால்', 'இந்

['நான்']
['என்', 'பெற்றோர்கள்', 'மிகவும்', 'ஆனால்', 'அவர்கள்', 'நம்பிக்கை', 'விட்டு', 'கொடுக்க', 'தொடங்கியது', 'கூட']
['நான்']
['நீ', 'மட்டும்', 'ஒரு', 'மாதத்திற்கு', 'இனி', 'நான்', 'விட', 'இந்த', 'குழுவினர்', 'இருந்து', 'வந்தது', 'அல்போன்ஸ்', 'நடக்கிறது', 'என்று', 'இந்த', 'கதை', 'என்ன']
['நான்']
['அவர்', 'ஒரு', 'வழக்கறிஞர்', 'மற்றும்', 'அவர்', 'ஒரு', 'அலுவலகத்தை', 'கீழே', 'இருக்கு', 'அவர்', 'சரி', 'ஒரு', 'பெரிய', 'பழைய', 'புணர்கிறேன்', 'மாளிகையை', 'கிடைத்தது']
['நான்']
['ஒரு', 'பாதுகாப்பான', 'இடத்தில்', 'நேரம்', 'இருக்கும்', 'போது', 'அந்த', 'நீங்கள்', 'அவர்களுக்கு', 'அடைய', 'முடியும்', 'அவர்கள்', 'அங்கு']
['நான்']
['பணக்கார', 'மற்றும்', 'பணக்கார', 'மற்றும்', 'என்', 'போனஸ்', 'பெரிய', 'பெரிய', 'எடுப்பாய்', 'இருக்கும்', 'மற்றும்', 'அதே', 'பழைய', 'கதை']
['நான்']
['அவர்', 'அதிர்ஷ்டசாலி', 'போதும்', 'இருந்தால்', 'அவர்', 'நீங்கள்', 'போன்ற', 'நின்று', 'வேண்டும்', 'என்று', 'அவன்', 'பழிவாங்க', 'வேண்டும்', 'என்று']
['நான்']
['இந்தர்', 'நீங்கள்', 'வரவில்லை', 'என்றால்', 'நான்', 'என்', 'சொந

['நான்']
['நீங்கள்', 'அதை', 'பற்றி', 'மீண்டும்', 'அங்கு', 'அனைத்து', 'என்ன', 'நரகத்தில்', 'என்னை', 'சொல்ல', 'இருக்காங்க']
['நான்']
['நான்', 'நீங்கள்', 'என்னை', 'மன்னிக்க', 'வேண்டும்', 'என்பதால்', 'நீங்கள்', 'இன்று', 'வந்து', 'நினைப்பார்கள்', 'பயம்']
['நான்']
['நாம்', 'ஒரு', 'டாலர்கள்', 'வேண்டும்', 'காதல்', 'தேடும்', 'இந்த', 'அலுவலகங்கள்', 'இருந்து', 'வெளியே', 'நடைபயிற்சி']
['நான்']
['நீங்கள்', 'என்னை', 'அனுமதிக்க', 'மாட்டேன்', 'என்றால்', 'நான்', 'எப்படி', 'உங்களுக்கு', 'உதவ', 'முடியும்']
['நான்']
['முக்கிய', 'ஒன்றும்', 'ஆனால்', 'நீங்கள்', 'இனி', 'அதை', 'எடுப்பாய்', 'இருக்கும்', 'நீ', 'அங்கே', 'பார்த்தேன்']
['நான்']
['சரி', 'அந்த', 'விசயத்துல', 'நான்', 'நீ', 'இங்க', 'வர', 'இந்த', 'ராக்']
['நான்']
['நாம்', 'வெள்ளை', 'மாளிகை', 'எல்லையை', 'பெற்று', 'விட்டேன்', 'மற்றும்', '10', 'தொகுதி', 'அகற்றப்படும்']
['நான்']
['உரிமையாளர்', 'திருமதி', 'லில்லி', 'அவர்', 'இந்த', 'பெயர்', 'அவரது', 'பிளாட்', 'மாற்றப்பட்டது']
['நான்']
['அவர்', 'என்னை', 'அங்கு', 'ஒரு', 'புத்தகம்', 'பெற', 'ஒரு', 'சில', 'உதவிக

['நான்']
['மிகவும்', 'என்ன', 'நான்', 'அதே', 'கடையில்', 'அல்லது', 'அதே', 'பயன்படுத்தப்படும்', 'என்றும்', 'உள்ளது']
['நான்']
['நான்', 'ஒரு', 'உறவு', 'இருந்தது', 'அவர்', 'வன்முறை', 'இருந்தது', 'அதனால்', 'இப்போது', 'நான்', 'இல்லை']
['நான்']
['உண்மையில்', 'நான்', 'நீங்கள்', 'சொல்ல', 'நான்', 'இங்கே', 'இருக்கிறேன்', 'சாம்', 'இந்த', 'மாலை', 'பார்க்கும்', 'போது']
['நான்']
['அடிக்கடி', 'கர்மா', 'எழுந்து', 'வந்து', 'உன்னை', 'பின்னால்', 'ஆனால்', 'நான்', 'அதை', 'பார்ப்பது', 'சில', 'முட்டைகளை', 'உடைக்க', 'வேண்டும்']
['நான்']
['நான்', 'காதல்', 'மற்றும்', 'திருமணத்தில்', 'நம்பிக்கை', 'மற்றும்', 'நான்', 'உண்மையில்', 'நீங்கள்', 'ஒரு', 'நல்ல', 'பையனை', 'நம்புகிறேன்']
['நான்']
['நான்', 'இன்னும்', 'நீங்கள்', 'இங்கு', 'எந்த', 'சட்ட', 'அதிகாரம்', 'கிடைத்தது', 'அதாவது', 'எந்த', 'தாள்', 'பார்க்க', 'வேண்டாம்']
['நான்']
['நாங்கள்', 'இன்னும்', 'மிக', 'இருக்கும்', 'பங்குகள்', 'சேமிக்கலாம்', 'பிறகும்', 'நாம்', 'இங்கு', 'இல்லை', 'கிடைத்துவிட்டது']
['நான்']
['நாம்', 'சொல்ல', 'பழைய', 'மனிதன்', 'முறைதவறி', 'பிறந்த'

['நான்']
['கீப்ஸ்', 'நல்ல', 'இருந்த', 'போது', 'நான்', 'பெற', 'வேண்டும்', 'வேறு', 'யாரையாவது', 'உங்கள்', 'கழுதை', 'பிணை', 'எடுக்க', 'கண்டுபிடிக்க']
['நான்']
['என்', 'மீட்பவர்', 'தான்', 'எழுப்பிய', 'கேள்விகள்', 'இருப்புஅச்சுறுத்தலானஇருக்கிறது', 'சிந்திக்க', 'வேண்டும்', 'ஆனால்நான்நான்கண்டேன்']
['நான்']
['நீ', 'கொடூரமான', 'விஷயங்களை', 'பார்த்து', 'வருகின்றனர்', 'ஆனால்', 'ஒவ்வொரு', 'கொண்டு', 'நீங்கள்', 'மீது', 'மழை', 'பார்க்க', 'வேண்டும்', 'என்று']
['நான்']
['ஓஹியோ', 'வேடிக்கையாக', 'மற்றும்', 'நீங்கள்', 'எப்போதும்', 'வழியாக', 'திரும்பி', 'வந்தால்', 'ஒருவேளை', 'நீங்கள்', 'எரிவாயு', 'ஐந்து', 'நிறுத்திவிடும்']
['நான்']
['நீ', 'எனக்கு', 'கீழ்', 'என்', 'செய்து', 'என்னை', 'அந்த', 'நான்', 'அழித்து', 'நீங்கள்', 'என்', 'எதிரிகள்', 'என்னை', 'தங்கள்', 'முதுகில்', 'திரும்ப', 'வேண்டும்']
['நான்']
['அவர்', 'தட்டி', 'தனது', 'கையை', 'நகர்த்த', 'முடியவில்லை', 'கடின', 'மிகவும்', 'நான்', 'ஒரு', 'பையன்', 'குத்தியதாக']
['நான்']
['எங்களைப்', 'பொறுத்தவரை', 'அது', 'ஒரு', 'சரணாலயம்', 'வெளி', 'உலக', 'இருந்து'

['நான்']
['நான்', 'அவரை', 'நம்', 'நாட்டின்', 'கிளப்', 'ஒரு', 'உறுப்பினர்', 'கிடைத்தது', 'இது', '90', '000', 'பிளஸ்', 'இருந்தது']
['நான்']
['நான்', 'என்', 'பணத்தை', 'உங்களுடன்', 'ஒரு', 'இரட்டை', 'மெருகூட்டல்', 'நிறுவனம்']
['நான்']
['ஒரு', 'கலங்கரை', 'விளக்கம்', 'இல்லாமல்', 'ஒரு', 'ஒரு', 'கண்டுபிடித்து', 'போல்', 'உள்ளது']
['நான்']
['சொற்கள்', 'இவை', 'என்று', 'நேரத்தில்', 'ஒரு', 'புள்ளியில்', 'இருந்து', 'வட்டி', 'சந்தை', 'வேண்டும்']
['நான்']
['நான்', 'ஒருவேளை', 'மட்டும்', 'விட்டு', 'வருகிறேன்', 'நான்', 'நல்ல', 'தயாராக', 'இருக்கிறேன்', 'போது']
['நான்']
['நாங்கள்', 'ஒன்றும்', 'செய்ய', 'வேண்டும்', 'ஆனால்', 'அவர்கள்', 'ஆஷெர்', 'கொல்ல', 'துணை', 'தலைவர்', 'மற்றும்', 'அவரது', 'முழு', 'தேசிய', 'பாதுகாப்பு', 'குழு']
['நான்']
['சரி', 'எனக்கு', 'இங்கு', 'ஒரு', 'காரை', 'தேடும்', 'பழைய', 'எல்', 'செய்']
['நான்']
['ஆனால்', 'எனக்கு', 'உன்னை', 'போல்', 'fellas', 'ஐந்து', 'காதல்', 'என்றால்', 'நீங்கள்', 'முடிந்தவரை', 'தொலைவில்', 'அவர்களிடம்', 'இருந்து', 'கிடைக்கும்']
['நான்']
['இது', 'பில்லி', 'சாத்திய

['நான்']
['அவர்', 'குழந்தை', 'ஜாக்', 'அவர்', 'ஐந்து', 'அவரது', 'சான்றுகள்', 'கோப்பு', 'கொடுத்தார்', 'எனவே', 'இது', 'அனைத்து', 'நாம்', 'ஒரு', 'செலுத்தி', 'அந்த']
['நான்']
['ஒரு', 'நான்', 'இந்த', 'பையன்', 'யார்', 'என்று', 'தெரியவில்லை', 'பாருங்கள்', 'அது', 'தான்', 'இருக்கிறது']
['நான்']
['நாங்கள்', 'அவள்', 'இன்னும்', 'மூச்சு', 'அவரது', 'அடக்கம்', 'சென்ற', 'போது', 'மட்டும்', 'தான்', 'இருந்தது']
['நான்']
['உங்களுக்கு', 'தெரியும்', 'நான்', 'உங்கள்', 'உயிரை', 'காப்பாற்ற', 'நான்', 'உங்கள்', 'சரி', 'கிடைக்கும்']
['நான்']
['நான்', 'உங்கள்', 'எங்கள்', 'இறுதி', 'திட்டம்', 'மூலம்', 'மகிழ்ச்சி', 'இருக்கும்', 'என்று', 'நான்', 'நம்புகிறேன்']
['நான்']
['மற்றும்', 'உங்கள்', 'குடும்பம்', 'ஏற்ப', 'நீங்கள்', 'மத', 'மற்றும்', 'கடவுள்']
['நான்']
['நாம்', 'மட்டும்', '100', 'டைனமைட்', 'என்ற', 'குச்சிகள்', 'மற்றும்', 'ஒரு', 'சில', 'ஆயுதங்கள்', 'கிடைத்தது']
['நான்']
['okay', 'ஆனால்', 'அதை', 'முப்பரிமாண', 'இரு', 'wormhole', 'ஒரு', 'வட்டம்']
['நான்']
['நான்', 'இப்போது', 'அழைப்பு', 'எடுக்க', 'கிடைக்க', 'வில்லை

['நான்']
['பாதுகாப்பாக', 'கட்டிடம்', 'வெளியே', 'அழைத்து', 'வருகின்றனர்', 'நான்', 'திரு', 'போபோ', 'மற்றும்', 'அவரது', 'மகள்', 'என்பதை', 'உறுதி', 'செய்ய', 'வேண்டும்']
['நான்']
['நீங்கள்', 'இந்த', 'நேரத்தில்', 'இப்போது', 'இது', 'சரியான', 'செய்ய', 'ஏன்', 'என்று', 'எனக்கு', 'புரியவில்லை']
['நான்']
['நீங்கள்', 'அவரை', 'உங்கள்', 'முழு', 'உடைந்த', 'சரங்களை', 'கட்டி', 'களிமண்', 'சீர்', 'செய்யப்படும்']
['நான்']
['நாள்', 'மூன்று', 'மூலம்', 'அவர்', 'ஏற்கனவே', 'எப்படியாவது', 'வரை', 'உட்கார்ந்து', 'மற்றும்', 'சுவரில்']
['நான்']
['எப்படி', 'நீங்கள்', 'எப்படி', 'பாலியல்', 'இணக்க', 'தெரியும்', 'நீங்கள்', 'அதை', 'ஒப்பிட்டு', 'இல்லை', 'போது']
['நான்']
['அதை', 'நீங்கள்', 'தெளிவான', 'குழந்தைகள்', 'முக்கிய', 'இருந்தன', 'நீங்கள்', 'நோய்', 'எதிர்ப்பு', 'என்பதால்']
['நான்']
['நீங்கள்', 'எனக்கு', 'நீங்கள்', 'உங்கள்', 'சொந்த', 'கொண்டு', 'வர', 'முடியாது', 'ஒரு', 'பொருட்டல்ல', 'உள்ளது']
['நான்']
['இரண்டு', 'வகையான', 'உள்ளன', 'இந்த', 'உலகத்தில்', 'உள்ள', 'மக்கள்', 'நடவடிக்கை', 'மற்றும்', 'அனைவரும்', 'ஆண்கள்

0.0

In [78]:
tamil_train[0]

'நாம் எங்கே போகிறோம் என்று.'

In [79]:
eng_train[0]

"That's where we're going."

In [80]:
Y_train[0]

[11, 63, 359, 7, 0]

In [82]:
processed_eng_train[0]

['that', "'s", 'where', 'we', "'re", 'going', '.']

[2, 15, 11, 102, 18, 32, 83, 4, 3]